# Setup and Configuration

This notebook sets up the configuration and initializes AWS/SageMaker sessions for the Two-Stage Recommendation System.


In [ ]:
import os
import boto3
import sagemaker
from sagemaker.feature_store.feature_store import FeatureStore
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configuration
def get_config():
    """Get configuration dictionary from environment variables."""
    return {
        'region': os.getenv('AWS_REGION', 'ap-south-1'),
        'role_arn': os.getenv('AWS_ROLE_ARN', ''),
        'bucket': os.getenv('S3_BUCKET', ''),
        'user_feature_group_name': os.getenv('USER_FEATURE_GROUP_NAME', ''),
        'item_feature_group_name': os.getenv('ITEM_FEATURE_GROUP_NAME', ''),
        'dynamodb_table_name': os.getenv('DYNAMODB_TABLE_NAME', 'user-candidates')
    }

def initialize_sessions(config: dict = None):
    """Initialize AWS and SageMaker sessions."""
    if config is None:
        config = get_config()
    
    # Initialize Boto3 session
    boto_session = boto3.Session(region_name=config['region'])
    
    # Create SageMaker session
    sagemaker_session = sagemaker.Session(
        boto_session=boto_session,
        default_bucket=config['bucket']
    )
    
    # Feature Store clients
    featurestore_runtime = boto_session.client(
        service_name='sagemaker-featurestore-runtime',
        region_name=config['region']
    )
    
    sagemaker_client = boto_session.client(
        service_name='sagemaker',
        region_name=config['region']
    )
    
    feature_store = FeatureStore(sagemaker_session=sagemaker_session)
    
    print(f"Initialized SageMaker session in {config['region']}")
    print(f"Default bucket: {config['bucket']}")
    
    return {
        'boto_session': boto_session,
        'sagemaker_session': sagemaker_session,
        'featurestore_runtime': featurestore_runtime,
        'sagemaker_client': sagemaker_client,
        'feature_store': feature_store,
        'config': config
    }

In [ ]:
# Get configuration
config = get_config()

In [ ]:
# Initialize sessions
sessions = initialize_sessions(config)

# Extract sessions for use in other notebooks
boto_session = sessions['boto_session']
sagemaker_session = sessions['sagemaker_session']
featurestore_runtime = sessions['featurestore_runtime']
sagemaker_client = sessions['sagemaker_client']
feature_store = sessions['feature_store']
